# Transformation Matrix Algorithm under various constraints

### The Problem Statement

Given two sets of $N$ points, the aim of this notebook is to find the 2D transformation matrix $M$ that can be applied to arrive from the original set to the transformed set.

### The Prerequisties

So what exactly is a 2D transformation matrix? A 2D transformation matrix is a 3x3 matrix that when applied to a point, transforms it to a different point in the cartesian plane. The last 3 elements of the 2D transformation matrix remain 0,0,1. Several different transformations are possible using these 6 numbers as shown in the figure below.

![](https://upload.wikimedia.org/wikipedia/commons/thumb/2/2c/2D_affine_transformation_matrix.svg/360px-2D_affine_transformation_matrix.svg.png)

Usually, when working with transformations, a mixture of the above transformations is involved. In this notebook, we are going to discuss solutions for the following subtypes:
1. Pure Translation
2. Pure Scaling
3. Scaling + Translation
4. Pure Rotation
5. Rotation + Translation
6. Scaling (with $S_{x} = S_{y}$) + Rotation + Translation (No shear)
7. Scaling + Rotation + Translation (No shear)
8. No constraints

###### We will be using the following notation for the same:

$p_{o}$ : Any Point from the original set of points (considered as the vector $[px, py]$ )

$p_{t}$ : Any Point from the transformed set of points (considered as the vector $[px, py]$ )

$ N $   : Number of points

$D_{x}$ : Translation in X direction

$D_{y}$ : Translation in Y direction

$\theta$  : Angle of rotation. 

$S_{x}$ : Scaling in X direction

$S_{y}$ : Scaling in Y direction

Since, we are not covering shear and reflection, we will be interchangably referring to transformation matrix $M$ as:

$\begin{bmatrix}M_{11} & M_{12} & M_{13}
\\ M_{21} & M_{22} & M_{23}\\ 0& 0 & 1\end{bmatrix} \Leftrightarrow \begin{bmatrix}S_{x} \cos{\theta} & S_{x} \sin{\theta} & D_{x}
\\ - S_{y} \sin{\theta} & S_{y}\cos{\theta} & D_{y}\\ 0& 0 & 1\end{bmatrix}$

### I ) Pure Translation
This is the easiest of all subtypes. Here,

$M = \begin{bmatrix}1 & 0 & D_{x}
\\ 0 & 1 & D_{y}\\ 0& 0 & 1\end{bmatrix}$

To solve this problem, computing a simple average on the vector $(P_{t}-P_{o})$ will give the values $D_{x}$ and $D_{y}$.

$D_{x} = \frac{\sum (px_{t}-px_{o})}{N}, D_{y} = \frac{\sum (py_{t}-py_{o})}{N}$

For **noisy** datapoints, we can compute the standard deviation over the lists $(px_{t}-px_{o})$ and $(py_{t}-py_{o})$ and choose a suitable deviation to remove the outliers as explained [here](#removing_outliers_using_std_dev). 

### II ) Pure Scaling
This is another easy problem to solve. Here,

$M = \begin{bmatrix}S_{x}  & 0 & 0
\\ 0 & S_{y}& 0\\ 0& 0 & 1\end{bmatrix}$

*Note: It is always assumed that the points are being scaled with respect to the origin.*

To solve this problem, we will compute a simple average on the vector $(P_{t} / P_{o})$. This will give the values $S_{x}$ and $S_{y}$.

$S_{x} = \frac{\sum (px_{t}/px_{o})}{N^{*}}, S_{y} = \frac{\sum (py_{t}/py_{o})}{N^{*}}$

*Note: For $px_{o} = 0$ and $py_{o} = 0$, we will be skipping the points from the average calculation and hence the need for $N^{*}$ which is a count of all points eventually considered.* 

For **noisy** datapoints, we can compute the standard deviation over the lists $(px_{t} / px_{o})$ and $(py_{t} / py_{o})$ and choose a suitable deviation to remove the outliers as explained [here](#removing_outliers_using_std_dev). 

### III ) Scaling + Translation
For this problem we will use,

$M = \begin{bmatrix}S_{x}  & 0 & D_{x}
\\ 0 & S_{y}& D_{y}\\ 0& 0 & 1\end{bmatrix}$

By writing this matrix as the solution, we are trying to imply that the points were first scaled and then translated. The order of operations is important because if we decide to assume that they were translated first and then scaled, we will have to use $S_{x}.D_{x}$ and $S_{y}.D_{y}$ instead of just $D_{x}$ and $D_{y}$ respectively. This is explained in more detail [here](#order_of_operations_and_why_it_matters). Nevertheless, regardless of the assumed order of operation, our final matrix will come out to be the same, so we will be using our Matrix M. 

$S_{x}.px_{o} + D_{x} = px_{t}$ and $S_{y}.py_{o} + D_{y} = py_{t}$

This is of the form $y = mx + c$ and we can use Linear Regression to solve this problem.


### IV ) Pure Rotation
Here,

$ M = \begin{bmatrix} \cos{\theta} &  \sin{\theta} & 0
\\ -  \sin{\theta} & \cos{\theta} & 0\\ 0& 0 & 1\end{bmatrix}$

To solve this problem, 


### V ) Rotation + Translation
Here,

$ M = \begin{bmatrix} \cos{\theta} &  \sin{\theta} & D_{x}
\\ -  \sin{\theta} & \cos{\theta} & D_{y}\\ 0& 0 & 1\end{bmatrix}$

To solve this problem, 


<a id='removing_outliers_using_std_dev'></a>

## Removing Outliers using *Standard deviation*

Let's assume a noisy dataset: $[3.2, 3.3, 3.5, 3.9, 4.1, 4.3, 4.7, 4.8, 15]$

If we compute a direct average, it will come out to be $5.19$. 

But we know that a noisy dataset can have outliers that affect the average values. After removing such outliers (in this case the value $15$, we get the true average to be $3.97$.

Here's a peek at a code to execute the same:


In [69]:
import numpy as np
noisy_list = [3.2, 3.3, 3.5, 3.9, 4.1, 4.3, 4.7, 4.8, 15]
print(np.average(noisy_list))

def remove_outliers(noisy_list, number_of_standard_deviations=2):
    std_dev = np.std(noisy_list)
    noise_free_list = []
    for x in noisy_list:
        if(np.abs(x-std_dev)<=number_of_standard_deviations*std_dev):
            noise_free_list.append(x)
    return noise_free_list

noise_free_list = remove_outliers(noisy_list)
print(np.average(noise_free_list))


5.199999999999999
3.9749999999999996


<a id='order_of_operations_and_why_it_matters'></a>

## Order of operations and why it matters

????????????